In [ ]:
import requests
import xml.etree.ElementTree as ET
import os
import re
import pickle
from dotenv import load_dotenv
import getpass

# Where to store session:
SESSION_FILE = ".balanz_session.pkl"

def save_session(access_token, cookie, id_cuenta):
    with open(SESSION_FILE, "wb") as f:
        pickle.dump((access_token, cookie, id_cuenta), f)

def load_session():
    if not os.path.exists(SESSION_FILE):
        return None, None, None
    try:
        with open(SESSION_FILE, "rb") as f:
            return pickle.load(f)
    except:
        return None, None, None

def login(username, password):
    # -- Get nonce --
    session = requests.Session()
    headers = {
        "User-Agent": "Mozilla/5.0",
        "Origin": "https://clientes.balanz.com",
        "Referer": "https://clientes.balanz.com/auth/login?avoidAuthRedirect=true",
        "Content-Type": "application/json"
    }
    # Get nonce
    resp = session.post("https://clientes.balanz.com/api/v1/auth/init?avoidAuthRedirect=true",
                        json={"user": username}, headers=headers)
    root = ET.fromstring(resp.text)
    nonce = root.findtext("nonce")
    if not nonce:
        raise Exception("Could not get nonce")
    # -- Post login XML --
    login_xml = f"""<jsonObject><user>{username}</user><pass>{password}</pass><nonce>{nonce}</nonce></jsonObject>"""
    headers["Content-Type"] = "application/xml"
    resp2 = session.post("https://clientes.balanz.com/api/v1/auth/login?avoidAuthRedirect=true",
                        data=login_xml, headers=headers)
    if "Sesiones activas" in resp2.text:
        raise Exception("Too many active sessions. Wait and try again.")
    root2 = ET.fromstring(resp2.text)
    access_token = root2.findtext("AccessToken")
    id_cuenta = root2.findtext("idPersona")  # Or correct field for idCuenta
    cookie_str = resp2.headers.get("Set-Cookie","")
    m = re.search(r'cookiesession1=([^;]+)', cookie_str)
    cookie = m.group(1) if m else None
    if not (access_token and cookie and id_cuenta):
        raise Exception("Login failed, no access/cookie/id")
    save_session(access_token, cookie, id_cuenta)
    return access_token, cookie, id_cuenta

def session_test_call(access_token, cookie, id_cuenta):
    # Try a lightweight authenticated call; here we use the "cotizacioninstrumento" endpoint as an example.
    url = "https://clientes.balanz.com/api/v1/cotizacioninstrumento"
    headers = {
        "Authorization": access_token,
        "Cookie": f"cookiesession1={cookie}",
        "Accept": "application/json",
        "User-Agent": "Mozilla/5.0"
    }
    params = {
        "plazo": "1",
        "idCuenta": id_cuenta,
        "ticker": "S2S5C"  # Put a valid, but simple/cheap ticker, e.g. known to always work
    }
    try:
        resp = requests.get(url, headers=headers, params=params, timeout=6)
        # If forbidden/expired
        if resp.status_code == 403 and "Sesion Expirada" in resp.text:
            return False
        if resp.status_code == 401:
            return False
        # If error codes in json
        try:
            data = resp.json()
            if "Sesion Expirada" in data.get("Descripcion",""):
                return False
        except Exception:
            pass
        # Otherwise session OK
        return True
    except:
        return False

def get_balanz_session(username, password):
    # 1. Try to use stored session first
    access_token, cookie, id_cuenta = load_session()
    if access_token and cookie and id_cuenta:
        ok = session_test_call(access_token, cookie, id_cuenta)
        if ok:
            print("Using previous session.")
            return access_token, cookie, id_cuenta
        else:
            print("Session expired or invalid, logging in again...")
    else:
        print("No stored session, logging in...")

    # 2. Do login, store session
    access_token, cookie, id_cuenta = login(username, password)
    print("New session obtained.")
    return access_token, cookie, id_cuenta

Nonce: 30006894-C900-4154-A023-6B7B96A661AD
Login response: <jsonObject><idError>-3</idError><Descripcion>Tiene demasiadas Sesiones activas. Cierre alguna sesión o intente nuevamente en 15 minutos</Descripcion></jsonObject>
AccessToken: None
Session cookie: None


In [ ]:
# Load .env variables into environment
load_dotenv()

# Read credentials
username = os.getenv("BALANZ_USER")
password = os.getenv("BALANZ_PASSWORD")

if not username:
    username = input("Please enter Balanz username: ")

if not password:
    password = getpass.getpass("Please enter Balanz password: ")

access_token, cookie, id_cuenta = get_balanz_session(username, password)
# You are now logged in and ready to use these credentials!

# Use access_token, cookie, id_cuenta for further API calls:
url = "https://clientes.balanz.com/api/v1/cotizacioninstrumento"
headers = {
    "Authorization": access_token,
    "Cookie": f"cookiesession1={cookie}",
    "Accept": "application/json",
    "User-Agent": "Mozilla/5.0"
}
params = {
    "plazo": "1",
    "idCuenta": id_cuenta,
    "ticker": "S2S5C"
}
resp = requests.get(url, headers=headers, params=params)
print(resp.status_code, resp.text)